# Introduction

...

In [1]:
# libraries to install
# !pip install selenium

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm # useful for progress bars
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import re
import os 

## 1. Data Collection

...

### 1.1. Get the list of animes

Here we will extract the *urls* and the *names* of the animes in the list (cfr. https://myanimelist.net/topanime.php). At first we can have an idea of the necessary steps to extract the informations we want by working on a single anime in the list and then proceed by iteration. 

After inspecting the HTML code of the site, we saw that the all the informations we need from a single anime are stored in  `tr` blocks inside a single `table` that contains the list of all the top animes in the site. To get the  name of an anime in the list we should work on `a` tags, whereas to get the url we need to work on `td` tags (leveraging the property `href`). 

Knowing these HTML details we can use the `selenium` library to do the web-scrapping.

In [3]:
from selenium.webdriver.chrome.service import Service

In [4]:
s = Service('/Users/dany/Desktop/adm-hw3/chromedriver')

In [5]:
# selenium with Chrome
driver = webdriver.Chrome(service=s)

In [6]:
# create a dataframe with links of each anime
df = pd.DataFrame(columns = ['Href'])

The following code was inspired by looking at the work that was done last year about https://www.goodreads.com, for example by https://github.com/GiorgiaSalvatori/ADM-HW3/blob/main/main.ipynb. Also the following post was useful https://towardsdatascience.com/how-to-use-selenium-to-web-scrape-with-example-80f9b23a843a.

In [7]:
# go page by page and and store links in a list
anime_list = []

for page in tqdm(range(0, 400)):
    url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:        
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                anime_list.append((link.contents[0], link.get('href')) )

100%|██████████| 400/400 [07:16<00:00,  1.09s/it]


In [8]:
# total number of animes
print(len(anime_list))

19124


In [9]:
# assign list to dataframe
df['Href'] = anime_list

In [10]:
# check for duplicates
df['Href'].nunique()

19124

It seems there is just one duplicate among the 19124 animes in the list. We think that a single repetition will not influence our analysis, so we leave the dataframe as it is.

In [11]:
# save dataframe into a csv file without header and comma separator. 
# this is equivalent to a txt file, but with also the names of the animes
# that can be of help in some data processing stages. 
df.to_csv('urls.csv',sep = ' ', header=False)

In [12]:
df['Href'].head()

0    (Fullmetal Alchemist: Brotherhood, https://mya...
1    (Gintama°, https://myanimelist.net/anime/28977...
2    (Shingeki no Kyojin Season 3 Part 2, https://m...
3    (Steins;Gate, https://myanimelist.net/anime/92...
4    (Fruits Basket: The Final, https://myanimelist...
Name: Href, dtype: object

We could also create a dictionary, this is useful in some circumnstances.

In [13]:
#keys
name = []   
#values
url = []    

for item in anime_list:
    name.append(item[0])
    url.append(item[1])
    
D = dict(zip(name, url))
print(len(list(D.values())[0:5]))

5


## 1.2. Crawl animes

We procede to:
- download the html corresponding to each of the collected urls;
- save its html in a file;
- organize the entire set of downloaded html pages into folders. Each folder will contain the htmls of the animes in page 1, page 2, ... of the list of animes.

To do so we extensively use the `os` library to create directories, changing paths, etc...

In [14]:
# remark: the execution can take quite some time

# returns current working directory
base = os.getcwd()  
t = 0
# we use the previously created dictionary D to get the urls we need
scrapped_urls = list(D.values())[0:-1]
for i in range(len(scrapped_urls)):
    if(i%50==0):
        # create a new folder
        # remark: the pages will start from 0
        page_identifier = i-(49*t)
        # subdirectory
        directory = f"page_{page_identifier}.html"
        # parent directories
        parent_dir = base
        # path
        path = os.path.join(parent_dir, directory)
        # make directory
        os.makedirs(path)
        # checkpoint
        print("Directory '%s' created" %directory)
        # change directory 
        os.chdir(path)
        t += 1
        
    # get urls
    URL = scrapped_urls[i]
    page = requests.get(URL)
    
    # parsing
    soup_data = BeautifulSoup(page.content, "html.parser")
    
    # saving
    with open(f"article_{i}.html", "w") as file:
        file.write(str(soup_data))
        
    # checkpoint
    print(f"Article {i} successfully written!")

Directory 'page_0.html' created
Article 0 successfully written!
Article 1 successfully written!
Article 2 successfully written!
Article 3 successfully written!
Article 4 successfully written!
Article 5 successfully written!
Article 6 successfully written!
Article 7 successfully written!
Article 8 successfully written!
Article 9 successfully written!
Article 10 successfully written!
Article 11 successfully written!
Article 12 successfully written!
Article 13 successfully written!
Article 14 successfully written!
Article 15 successfully written!
Article 16 successfully written!
Article 17 successfully written!
Article 18 successfully written!
Article 19 successfully written!
Article 20 successfully written!
Article 21 successfully written!
Article 22 successfully written!
Article 23 successfully written!
Article 24 successfully written!
Article 25 successfully written!
Article 26 successfully written!
Article 27 successfully written!
Article 28 successfully written!
Article 29 successful

Article 240 successfully written!
Article 241 successfully written!
Article 242 successfully written!
Article 243 successfully written!
Article 244 successfully written!
Article 245 successfully written!
Article 246 successfully written!
Article 247 successfully written!
Article 248 successfully written!
Article 249 successfully written!
Directory 'page_5.html' created
Article 250 successfully written!
Article 251 successfully written!
Article 252 successfully written!
Article 253 successfully written!
Article 254 successfully written!
Article 255 successfully written!
Article 256 successfully written!
Article 257 successfully written!
Article 258 successfully written!
Article 259 successfully written!
Article 260 successfully written!
Article 261 successfully written!
Article 262 successfully written!
Article 263 successfully written!
Article 264 successfully written!
Article 265 successfully written!
Article 266 successfully written!
Article 267 successfully written!
Article 268 succ

Article 478 successfully written!
Article 479 successfully written!
Article 480 successfully written!
Article 481 successfully written!
Article 482 successfully written!
Article 483 successfully written!
Article 484 successfully written!
Article 485 successfully written!
Article 486 successfully written!
Article 487 successfully written!
Article 488 successfully written!
Article 489 successfully written!
Article 490 successfully written!
Article 491 successfully written!
Article 492 successfully written!
Article 493 successfully written!
Article 494 successfully written!
Article 495 successfully written!
Article 496 successfully written!
Article 497 successfully written!
Article 498 successfully written!
Article 499 successfully written!
Directory 'page_10.html' created
Article 500 successfully written!
Article 501 successfully written!
Article 502 successfully written!
Article 503 successfully written!
Article 504 successfully written!
Article 505 successfully written!
Article 506 suc

Article 715 successfully written!
Article 716 successfully written!
Article 717 successfully written!
Article 718 successfully written!
Article 719 successfully written!
Article 720 successfully written!
Article 721 successfully written!
Article 722 successfully written!
Article 723 successfully written!
Article 724 successfully written!
Article 725 successfully written!
Article 726 successfully written!
Article 727 successfully written!
Article 728 successfully written!
Article 729 successfully written!
Article 730 successfully written!
Article 731 successfully written!
Article 732 successfully written!
Article 733 successfully written!
Article 734 successfully written!
Article 735 successfully written!
Article 736 successfully written!
Article 737 successfully written!
Article 738 successfully written!
Article 739 successfully written!
Article 740 successfully written!
Article 741 successfully written!
Article 742 successfully written!
Article 743 successfully written!
Article 744 su

Article 953 successfully written!
Article 954 successfully written!
Article 955 successfully written!
Article 956 successfully written!
Article 957 successfully written!
Article 958 successfully written!
Article 959 successfully written!
Article 960 successfully written!
Article 961 successfully written!
Article 962 successfully written!
Article 963 successfully written!
Article 964 successfully written!
Article 965 successfully written!
Article 966 successfully written!
Article 967 successfully written!
Article 968 successfully written!
Article 969 successfully written!
Article 970 successfully written!
Article 971 successfully written!
Article 972 successfully written!
Article 973 successfully written!
Article 974 successfully written!
Article 975 successfully written!
Article 976 successfully written!
Article 977 successfully written!
Article 978 successfully written!
Article 979 successfully written!
Article 980 successfully written!
Article 981 successfully written!
Article 982 su

...

Article 18026 successfully written!
Article 18027 successfully written!
Article 18028 successfully written!
Article 18029 successfully written!
Article 18030 successfully written!
Article 18031 successfully written!
Article 18032 successfully written!
Article 18033 successfully written!
Article 18034 successfully written!
Article 18035 successfully written!
Article 18036 successfully written!
Article 18037 successfully written!
Article 18038 successfully written!
Article 18039 successfully written!
Article 18040 successfully written!
Article 18041 successfully written!
Article 18042 successfully written!
Article 18043 successfully written!
Article 18044 successfully written!
Article 18045 successfully written!
Article 18046 successfully written!
Article 18047 successfully written!
Article 18048 successfully written!
Article 18049 successfully written!
Directory 'page_361.html' created
Article 18050 successfully written!
Article 18051 successfully written!
Article 18052 successfully wri

Article 18251 successfully written!
Article 18252 successfully written!
Article 18253 successfully written!
Article 18254 successfully written!
Article 18255 successfully written!
Article 18256 successfully written!
Article 18257 successfully written!
Article 18258 successfully written!
Article 18259 successfully written!
Article 18260 successfully written!
Article 18261 successfully written!
Article 18262 successfully written!
Article 18263 successfully written!
Article 18264 successfully written!
Article 18265 successfully written!
Article 18266 successfully written!
Article 18267 successfully written!
Article 18268 successfully written!
Article 18269 successfully written!
Article 18270 successfully written!
Article 18271 successfully written!
Article 18272 successfully written!
Article 18273 successfully written!
Article 18274 successfully written!
Article 18275 successfully written!
Article 18276 successfully written!
Article 18277 successfully written!
Article 18278 successfully w

Article 18475 successfully written!
Article 18476 successfully written!
Article 18477 successfully written!
Article 18478 successfully written!
Article 18479 successfully written!
Article 18480 successfully written!
Article 18481 successfully written!
Article 18482 successfully written!
Article 18483 successfully written!
Article 18484 successfully written!
Article 18485 successfully written!
Article 18486 successfully written!
Article 18487 successfully written!
Article 18488 successfully written!
Article 18489 successfully written!
Article 18490 successfully written!
Article 18491 successfully written!
Article 18492 successfully written!
Article 18493 successfully written!
Article 18494 successfully written!
Article 18495 successfully written!
Article 18496 successfully written!
Article 18497 successfully written!
Article 18498 successfully written!
Article 18499 successfully written!
Directory 'page_370.html' created
Article 18500 successfully written!
Article 18501 successfully wri

Article 18700 successfully written!
Article 18701 successfully written!
Article 18702 successfully written!
Article 18703 successfully written!
Article 18704 successfully written!
Article 18705 successfully written!
Article 18706 successfully written!
Article 18707 successfully written!
Article 18708 successfully written!
Article 18709 successfully written!
Article 18710 successfully written!
Article 18711 successfully written!
Article 18712 successfully written!
Article 18713 successfully written!
Article 18714 successfully written!
Article 18715 successfully written!
Article 18716 successfully written!
Article 18717 successfully written!
Article 18718 successfully written!
Article 18719 successfully written!
Article 18720 successfully written!
Article 18721 successfully written!
Article 18722 successfully written!
Article 18723 successfully written!
Article 18724 successfully written!
Article 18725 successfully written!
Article 18726 successfully written!
Article 18727 successfully w

Article 18924 successfully written!
Article 18925 successfully written!
Article 18926 successfully written!
Article 18927 successfully written!
Article 18928 successfully written!
Article 18929 successfully written!
Article 18930 successfully written!
Article 18931 successfully written!
Article 18932 successfully written!
Article 18933 successfully written!
Article 18934 successfully written!
Article 18935 successfully written!
Article 18936 successfully written!
Article 18937 successfully written!
Article 18938 successfully written!
Article 18939 successfully written!
Article 18940 successfully written!
Article 18941 successfully written!
Article 18942 successfully written!
Article 18943 successfully written!
Article 18944 successfully written!
Article 18945 successfully written!
Article 18946 successfully written!
Article 18947 successfully written!
Article 18948 successfully written!
Article 18949 successfully written!
Directory 'page_379.html' created
Article 18950 successfully wri